In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

In [3]:
df = pd.read_csv('./data/csv-files/2005_brooklyn.csv')

In [5]:
df

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,3,BATH BEACH,01 ONE FAMILY HOMES,1,6362,13,,A1,65 BAY 11TH STREET,,...,1,0,1,2904,2212,1899,1,A1,0,2005-01-11
1,3,BATH BEACH,01 ONE FAMILY HOMES,1,6362,13,,A1,65 BAY 11TH STREET,,...,1,0,1,2904,2212,1899,1,A1,10,2005-01-11
2,3,BATH BEACH,01 ONE FAMILY HOMES,1,6362,21,,A1,51 BAY 11 STREET,,...,1,0,1,2604,2122,1899,1,A1,600000,2005-10-05
3,3,BATH BEACH,01 ONE FAMILY HOMES,,6362,61,,,8642 16 AVENUE,,...,0,0,0,0,0,0,1,A9,0,2005-12-27
4,3,BATH BEACH,01 ONE FAMILY HOMES,,6362,61,,,8642 16 AVENUE,,...,0,0,0,0,0,0,1,A9,745000,2005-11-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34238,3,WYCKOFF HEIGHTS,29 COMMERCIAL GARAGES,4,3407,34,,G9,950 WYCKOFF AVENUE,,...,0,1,1,7200,3000,1925,4,G9,0,2005-10-27
34239,3,WYCKOFF HEIGHTS,30 WAREHOUSES,4,3177,2,,E9,17 WYCKOFF AVENUE,,...,0,1,1,2357,4713,1931,4,E9,0,2005-12-08
34240,3,WYCKOFF HEIGHTS,31 COMMERCIAL VACANT LAND,4,3345,25,,V1,1538 MYRTLE AVENUE,,...,0,0,0,4859,0,0,4,V1,0,2005-02-09
34241,3,WYCKOFF HEIGHTS,41 TAX CLASS 4 - OTHER,4,3188,30,,V1,386 TROUTMAN STREET,,...,0,0,0,2500,0,0,4,Z9,0,2005-11-25
